# 1. Configurar ambiente

In [ ]:
#Instalando bibliotecas
import pandas as pd
import re, os, random, pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import jellyfish
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import torch

#Definiendo variables del proyecto:
nlp = spacy.load('es_core_news_md')

# 2. Importar verbos

In [ ]:
# Importar la lista_verbos:
lista_verbos = 'tu algoritmo'

# Importar el diccionario:
verbos_irregulares = 'tu algoritmo'

# 3. Tratamiento de datos

In [ ]:
#Función para encontrar la raiz de las palabras
def raiz(palabra):
  palabra_encontrada = 'tu algoritmo'
  return palabra_encontrada

def tratamiento_texto(texto):
  texto = 'tu algoritmo'
  return texto

#Función para reemplazar el final de una palabra por 'r'
def reemplazar_terminacion(palabra):
  nueva_palabra = 'tu algoritmo'
  return nueva_palabra.split()[0]

#Función para devolver los tokens normalizados del texto
def normalizar(texto):
  tokens=[]
  doc = nlp(texto)
  for t in doc:
    lemma=verbos_irregulares.get(t.text, t.lemma_.split()[0])
    lemma=re.sub(r'[^\w\s+\-*/]', '', lemma)
    if t.pos_ in ('VERB','PROPN','PRON','NOUN','AUX','SCONJ','ADJ','ADV','NUM') or lemma in lista_verbos:
      if t.pos_=='VERB':
        lemma = reemplazar_terminacion(lemma)
        tokens.append(raiz(tratamiento_texto(lemma)))
      else:
        tokens.append(tratamiento_texto(lemma))

  tokens = list(dict.fromkeys(tokens))
  tokens = list(filter(None, tokens))
  return tokens

# 4. Cargar bases de documentos

In [ ]:
#Importando bases de dialogo fluído
txt_folder_path = folder+'/dialogos'
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".txt")]
lista_dialogos, lista_dialogos_respuesta, lista_tipo_dialogo = [],[],[]
for idx in range(len(lista_documentos)):
  f=open(txt_folder_path+'/'+lista_documentos[idx], 'r', encoding='utf-8', errors='ignore')
  for line in f.read().split('\n'):
    lista_dialogos = 'Tu algoritmo aqui'
    lista_tipo_dialogo = 'Tu algoritmo aqui'
    lista_dialogos_respuesta = 'Tu algoritmo aqui'

#Creando Dataframe de diálogos
datos = {'dialogo':lista_dialogos,'respuesta':lista_dialogos_respuesta,'tipo':lista_tipo_dialogo,'interseccion':0,'jaro_winkler':0,'probabilidad':0}
df_dialogo = pd.DataFrame(datos)
df_dialogo = df_dialogo.drop_duplicates(keep='first')
df_dialogo.reset_index(drop=True, inplace=True)

# 5. Buscar respuesta del Chatbot

In [ ]:
#Función para verificar si el usuário inició un diálogo
def dialogo(user_response):
  user_response = tratamiento_texto(user_response) #Tratando el texto
  user_response = re.sub(r"[^\w\s]", '', user_response) #Elimina signos de puntuación
  df = df_dialogo.copy()
  for idx,row in df.iterrows():
    df.at[idx,'interseccion'] = "tu algoritmo aqui metodo1(user_response, row['dialogo'])"
    df.at[idx,'similarity'] = "tu algoritmo aqui metodo2(user_response, row['dialogo'])"
    df.at[idx,'jaro_winkler'] = "tu algoritmo aqui metodo3(user_response, row['dialogo'])"
    df.at[idx,'probabilidad'] = max(df.at[idx,'interseccion'],df.at[idx,'similarity'],df.at[idx,'jaro_winkler'])
  df.sort_values(by=['probabilidad','jaro_winkler'], inplace=True, ascending=False)
  probabilidad = df['probabilidad'].head(1).values[0]
  if probabilidad >= 0.93:
    print('Respuesta encontrada por el método de comparación de textos - Probabilidad: ', probabilidad)
    respuesta = df['respuesta'].head(1).values[0]
  else:
    respuesta = ''
  return respuesta

#Cargar tu modelo entrenado aqui(recuerda siempre cargar el modelo y el vectorizer o tokenizer usado en el entrenamiento del modelo):
ruta_modelo = folder+'/modelo'
Modelo_TF = BertForSequenceClassification.from_pretrained(ruta_modelo)
tokenizer_TF = BertTokenizer.from_pretrained(ruta_modelo)

#Función para dialogar utilizando el modelo
def clasificacion_modelo(pregunta):
  frase = ' '.join(normalizar(pregunta))
  clase_encontrada = 'Realiza la predicción de la frase con tu modelo aqui'

  #Buscar respuesta más parecida en la clase encontrada
  df = df_dialogo[df_dialogo['tipo'] == clase_encontrada]
  df.reset_index(inplace=True)
  vectorizer = TfidfVectorizer()
  dialogos_num = vectorizer.fit_transform(df['dialogo'])
  pregunta_num = vectorizer.transform([tratamiento_texto(pregunta)])
  similarity_scores = cosine_similarity(dialogos_num, pregunta_num)
  indice_pregunta_proxima = similarity_scores.argmax()

  if max(similarity_scores)>0.5 and clase_encontrada not in ['Otros']:
    print('Respuesta encontrada por el modelo Transformers - tipo:',clase_encontrada)
    respuesta = df['respuesta'][indice_pregunta_proxima]
  else:
    respuesta = ''
  return respuesta

#Función para devolver una respuesta final buscada en todos los métodos disponibles
def respuesta_chatbot(pregunta):
  respuesta = dialogo(pregunta)
  if respuesta != '':
    return respuesta
  else:
    respuesta = clasificacion_modelo(pregunta)
    if respuesta != '':
      return respuesta
    else:
      return 'Respuesta no encontrada'

# 6. Ejecutar Chatbot

In [ ]:
pregunta='CÓMO ESTAS'
respuesta = respuesta_chatbot(pregunta)
print(respuesta)

Respuesta encontrada por el modelo Transformers - tipo: Saludos
Estoy bien, gracias. ¿Y tú?
